##### Importing Required Libraries

In [17]:
import os
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import FewShotChatMessagePromptTemplate

from dotenv import load_dotenv

##### Large Language Model (LLM) 

In [18]:
load_dotenv(dotenv_path = r'C:..\LangChain\Examples\.env') #Path to you .env file
llm = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],  
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0.6,
)



In [19]:
print(os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"))
print(os.environ.get("AZURE_OPENAI_API_VERSION"))

gpt-35-turbo
2023-03-15-preview


In [20]:
# using invoke method
llm.invoke(
    "What are some fruits that are red in colour?"
)

AIMessage(content='Some fruits that are red in color are:\n\n1. Strawberry\n2. Raspberry\n3. Cherry\n4. Cranberry\n5. Watermelon\n6. Pomegranate\n7. Red apple\n8. Red grapefruit\n9. Red pear\n10. Red plum', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 18, 'total_tokens': 76, 'completion_tokens_details': None}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-845be476-4e3b-4d74-b4cc-8bbca52ba3d3-0', usage_metadata={'input_tokens': 18, 'output_tokens': 58, 'total_tokens': 76})

In [21]:
# using stream method
for chunk in llm.stream(
    "What are some fruits that are red in colour?"
):
    print(chunk, end="", flush=True)

content='' additional_kwargs={} response_metadata={} id='run-7daefbba-6389-45a6-bf1c-4d0935cd2590'content='Some' additional_kwargs={} response_metadata={} id='run-7daefbba-6389-45a6-bf1c-4d0935cd2590'content=' fruits' additional_kwargs={} response_metadata={} id='run-7daefbba-6389-45a6-bf1c-4d0935cd2590'content=' that' additional_kwargs={} response_metadata={} id='run-7daefbba-6389-45a6-bf1c-4d0935cd2590'content=' are' additional_kwargs={} response_metadata={} id='run-7daefbba-6389-45a6-bf1c-4d0935cd2590'content=' red' additional_kwargs={} response_metadata={} id='run-7daefbba-6389-45a6-bf1c-4d0935cd2590'content=' in' additional_kwargs={} response_metadata={} id='run-7daefbba-6389-45a6-bf1c-4d0935cd2590'content=' color' additional_kwargs={} response_metadata={} id='run-7daefbba-6389-45a6-bf1c-4d0935cd2590'content=' are' additional_kwargs={} response_metadata={} id='run-7daefbba-6389-45a6-bf1c-4d0935cd2590'content=':\n\n' additional_kwargs={} response_metadata={} id='run-7daefbba-6389-4

In [22]:
# using batch method
llm.batch(
    [
         "What are some fruits that are red in colour?",
          "What are some fruits that are blue in colour?"
    ]
)

[AIMessage(content='Some fruits that are red in color are strawberries, raspberries, cherries, watermelons, tomatoes, red apples, cranberries, pomegranates, and red grapes.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 18, 'total_tokens': 55, 'completion_tokens_details': None}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-f7f22eb3-d819-41eb-988a-faa987dc4ecd-0', usage_metadata={'input_tokens': 18, 'output_tokens': 37, 'total_tokens': 55}),
 AIMessage(content='Some fruits that are blue in color are blueberries, blackberries, and elderberries.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 18, 'total_tokens': 36, 'completion_tokens_details': None}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-0

##### Prompts Template

By default, PromptTemplate uses Python's str.format syntax for templating.

In [23]:
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

The template supports any number of variables, including no variables:

In [24]:
prompt_template = PromptTemplate.from_template("Tell me a joke")
prompt_template.format()

'Tell me a joke'

The prompt to chat models/ is a list of chat messages.

In [25]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
messages = chat_template.format_messages(text="I don't like eating tasty things")
print(messages)

response = llm.invoke(messages)
print(response)

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat.", additional_kwargs={}, response_metadata={}), HumanMessage(content="I don't like eating tasty things", additional_kwargs={}, response_metadata={})]
content='Let\'s try to put a positive spin on that! How about: "I enjoy exploring new and healthy food options that nourish my body and make me feel great!"' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 38, 'total_tokens': 72, 'completion_tokens_details': None}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}} id='run-f04f5651-1325-4e6e-9304-f34709dac69f-0' usage_metadata={'input_tokens': 38, 'output_tokens': 34, 'total_tokens': 72}


Few-shot examples for chat models

In [26]:
# define the examples you'd like to include
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())


Human: 2+2
AI: 4
Human: 2+3
AI: 5


In [27]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)


In [28]:

chain = final_prompt | llm

chain.invoke({"input": "What's the square of a triangle?"})

AIMessage(content="A triangle does not have a square. However, the area of a triangle can be calculated using the formula A = 1/2 * base * height, where base is the length of the triangle's base and height is the perpendicular distance from the base to the opposite vertex.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 58, 'total_tokens': 114, 'completion_tokens_details': None}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-e034eabd-6fcd-4270-8b02-90f83e5c61bc-0', usage_metadata={'input_tokens': 58, 'output_tokens': 56, 'total_tokens': 114})

##### Memory -- ConversationBufferMemory Example

In [36]:

from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_core.prompts import MessagesPlaceholder


# Define your prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a chatbot having a conversation with a human."
        ),  # The persistent system prompt
        MessagesPlaceholder(
            variable_name="chat_history"
        ),  # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),  # Where the human input will injected
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


# Create the LLMChain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=True
    
)

# Example user input
llm_chain.predict(human_input="Hi there my friend")



Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: Hi there my friend

> Finished chain.


'Hello! How can I assist you today?'

In [37]:
llm_chain.predict(human_input="Not too bad - how are you?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: Hi there my friend
AI: Hello! How can I assist you today?
Human: Not too bad - how are you?

> Finished chain.


"As an AI language model, I don't have feelings, but I'm functioning properly and ready to assist you with any questions or concerns you may have. How can I help you today?"

In [38]:
llm_chain.predict(human_input="what was my first interaction with you?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: Hi there my friend
AI: Hello! How can I assist you today?
Human: Not too bad - how are you?
AI: As an AI language model, I don't have feelings, but I'm functioning properly and ready to assist you with any questions or concerns you may have. How can I help you today?
Human: what was my first interaction with you?

> Finished chain.


"I'm sorry, but I don't have the capability to remember previous interactions with individual users, as I am designed to have conversations with multiple users. However, I'm here to help you with any questions or concerns you may have today."

##### Simple Agent

In [39]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.tools import Tool
from serpapi import GoogleSearch  

# Function for searching using SerpAPI
def search_google(query):
    params = {
        "q": query,
        "api_key": "enter your api key here"
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    return results['organic_results']

# Wrap the search tool in Langchain's Tool interface
search_tool = Tool(
    name="GoogleSearch",
    func=search_google,
    description="A tool to search the web for information.",

)

# Load the tool into Langchain
tools = [search_tool]

# Initialize the agent with the tool
agent = initialize_agent(
    tools=tools,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=llm,
    
    
)

# Example query to the agent
query = "What is the latest news about AI in healthcare?"

# Run the agent
response = agent.run(query)

# Output the response
print(response)

The latest news about AI in healthcare can be found on Healthcare IT News.
